In [1]:
# Parth korat
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import math

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\parth\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


In [3]:
# Visit the URL with San Diego County zip codes
url = 'https://www.zillow.com/browse/homes/ca/san-diego-county/'
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
# find 'ul' tag with the class 'bh-body-links' tag that contains all zip codes
ul =soup.find('ul', class_='bh-body-links')

sd_zip_codes = []

# find the 'li' tag and get all children under the tag
zip_code_tags = ul.findChildren('li', recursive=False)

# run a for loop to get all zip codes, then append to zip_codes list
for zip_code_tag in zip_code_tags:
    sd_zip_codes.append(zip_code_tag.text.strip('\n'))

# show first 5 zip codes from the scrape
sd_zip_codes[:5]

['91910', '92101', '92126', '91911', '92592']

In [6]:
# create dictionary with the 'San Diego County Zip Codes' as the key and the zip_codes list as the values
dict_zip_codes = {'San Diego County Zip Codes': sd_zip_codes}

# read the dictionary as a DataFrame object
zip_codes_df = pd.DataFrame(dict_zip_codes)

len(zip_codes_df)

192

In [7]:
start = 'https://www.greatschools.org/california/schools/?gradeLevels%5B%5D=e&gradeLevels%5B%5D=m&gradeLevels%5B%5D=h&gradeLevels%5B%5D=p&st%5B%5D=public_charter&st%5B%5D=public&st%5B%5D=charter&tableView=Overview&view=table'
loop_page = start + '&page={}'

browser.visit(start)

In [8]:
html_2 = browser.html
soup_2 = bs(html_2, 'html.parser')

In [9]:
# find the number of pages for scrapping
all_schools = int(soup_2.find('div', class_='pagination-summary').text.split(' ')[5].replace(',', ''))
pages = int(math.ceil(all_schools / 25))

In [10]:
# initalize the dictionary to hold the values
schools = {
    'School Name': [],
    'School Rating': [],
    'School Address': [],
    'Zip Code': [],
    'Grades': [],
    'Total Students Enrolled': [],
    'Students Per Teacher': [],
    'District': []
}

In [11]:
for page in range(1, pages):

    browser.visit(loop_page.format(page))
    soup_page = bs(browser.html, 'html.parser')
        
    # find the 'table' tag that includes all schools
    table = soup_page.find('table')

    # get the 'tbody' tag that is a child of the table tag
    tbody = table.findChild('tbody')

    # within the tbody tag there are multiple 'tr' tags that contain one school each on that page
    school_tags = tbody.findChildren('tr', recursive=False)
    
    # loop through the list and append the zip code from address to a list
    for school_tag in school_tags:

        # core includes the name, rating, and address of the school
        core = school_tag.findChildren('td')[0]

        # find name in the core
        name = core.find('a', class_='name').text
        
        # find rating in the core
        rating = core.find('span', class_='tipso_style').text.split('/')[0]
        
        # find address in the core
        address = core.find('div', class_='address').text

        # find rest of the data in the table by indexing
        district_type = school_tag.findChildren('td')[1].text
        grades = school_tag.findChildren('td')[2].text
        num_students = school_tag.findChildren('td')[3].text
        stu_teach_ratio = school_tag.findChildren('td')[4].text
        district = school_tag.findChildren('td')[6].text

        # strip the address by the comma and get the last item in the list to get the zip
        current_zip = address.split(',')[-1].strip(' ')

        # clean address
        mod_address = address.replace(', CA, ' + current_zip,'')
        
        if current_zip in sd_zip_codes and rating != 'Currently unrated':
            schools['School Name'].append(name)
            schools['School Rating'].append(rating)
            schools['School Address'].append(mod_address)
            schools['Zip Code'].append(current_zip)
            schools['Grades'].append(grades)
            schools['Total Students Enrolled'].append(num_students)
            schools['Students Per Teacher'].append(stu_teach_ratio)
            schools['District'].append(district)
        else:
            exit()

public_schools = pd.DataFrame(schools)
public_schools

,School Name,School Rating,School Address,Zip Code,Grades,Total Students Enrolled,Students Per Teacher,District
0,Maac Community Charter School,1,"1385 Third Avenue, Chula Vista",91911,9-12,201,21:1,Sweetwater Union High
1,SIATech School,1,"2611 Temple Heights Drive, Suite A, Oceanside",92056,11-12,1137,22:1,Vista Unified
2,Urban Corps of San Diego County Charter,1,"3127 Jefferson Street, San Diego",92110,9-12,285,20:1,Mono County Office of Education
3,Diego Hills Central Public Charter,1,"4585 College Avenue, San Diego",92115,K-12,748,19:1,Dehesa Elementary
4,Audeo Charter School III,1,"200 East Via Rancho Parkway, Escondido",92025,6-12,88,N/A,Audeo Charter School III District
5,Campo High (Continuation),1,"3305 Buckman Springs Road, Pine Valley",91962,9-12,24,11:1,Mountain Empire Unified
6,Alta Vista High (Continuation),1,"1575 Bonair Road, Vista",92084,9-12,113,8:1,Vista Unified
